In [ ]:
# -*- coding: utf-8 -*-
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import logging
import os
import re

# 로깅 설정 (노트북 출력 및 콘솔에 표시됨)
# 기존 핸들러 제거 (노트북에서 재실행 시 중복 로깅 방지)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 요청 헤더 (실제 브라우저처럼 보이도록 설정)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Wanted-Platform': 'web', # 필요할 수 있는 헤더
    'Wanted-Service': 'wanted', # 필요할 수 있는 헤더
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7', # 언어 설정 추가
}

logging.info("라이브러리 임포트, 로깅 및 헤더 설정 완료")

In [ ]:
# 직종 구분 선택
job_category = "total" # "total" # "backend" # "frontend"

if job_category == "total":
    job_category_url = "?"
elif job_category == "backend":
    job_category_url = "?jobCategory=1&"
elif job_category == "frontend":
    job_category_url = "?jobCategory=2&"
else:
    job_category_url = "?"


In [ ]:
# skill 데이터 필터링 함수 정의
def filter_skill_data(skill):
    """
    skill 데이터에서 조건부로 특수문자를 제거하고, 단어 목록 형태로 정리합니다.
    """
    if not skill:
        return ""

    # 0. 한글 제거
    no_hangul = re.compile('[ㄱ-ㅣ가-힣]+')
    filtered_skill = no_hangul.sub('', skill)

    # 1. 개행 문자 제거
    filtered_skill = filtered_skill.replace('\n', '')

    # 2. LINE SEPARATOR 제거 (U+2028)
    filtered_skill = filtered_skill.replace('\u2028', '')

    # # 3. 특수문자 중 '#', '+'를 제외하고 제거 (온점 포함)
    # # 제거 대상: 알파벳, 숫자, '#', '+', 공백이 아닌 모든 문자 (온점 포함)
    # filtered_skill = re.sub(r"[^a-zA-Z0-9#+\s]", "", filtered_skill)
    

    # 제거 대상: 알파벳, '#', '+', 공백이 아닌 모든 문자 (온점 포함)
    filtered_skill = re.sub(r"[^a-zA-Z#+\s]", "", filtered_skill)


    # 4. 알파벳 오른쪽 옆에 공백 없이 붙어 있는 숫자를 제외한 모든 숫자 제거
    def remove_standalone_numbers(text):
        def replace(match):
            return ""
        # 숫자 앞뒤로 알파벳이 없는 경우 제거
        return re.sub(r"(?<![a-zA-Z])\d+(?![a-zA-Z])", replace, text)

    filtered_skill = remove_standalone_numbers(filtered_skill)

    # 5. 단어 분리, 공백 제거, 중복 제거 및 ', '로 연결
    words = filtered_skill.split()  # 공백을 기준으로 단어 분리
    unique_words = []
    seen = set()
    for word in words:
        if word not in seen:
            unique_words.append(word)
            seen.add(word)

    filtered_skill = ', '.join(unique_words)

    return filtered_skill

In [ ]:
def scrape_jumpit_positions():
    """Jumpit 채용 공고를 스크래핑하여 데이터 리스트를 반환합니다."""
    all_positions_data = []
    page_no = 1

    logging.info("Jumpit 채용 공고 스크래핑 시작")

    while True:
        url = f"https://jumpit-api.saramin.co.kr/api/positions{job_category_url}sort=rsp_rate&highlight=false&page={page_no}"
        logging.info(f"페이지 {page_no}의 JSON 데이터 요청: {url}")

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()

            data = response.json()

            positions_list = data['result']['positions']

            if not positions_list:
                logging.info("더 이상 채용 공고가 없습니다. 스크래핑 종료.")
                break

            logging.info(f"페이지 {page_no}에서 {len(positions_list)}개의 채용 공고 발견")

            for position in positions_list:
                company_name = position['companyName']
                position_title = position['title']
                tech_stacks = position['techStacks']
                skill = ', '.join(tech_stacks) if tech_stacks else "N/A"

                # --- 여기에 필터링 함수 적용 ---
                filtered_skill = filter_skill_data(skill)
                # --- 필터링 완료 ---

                position_data = {
                    'company': company_name,
                    'position': position_title,
                    'skill': filtered_skill,
                }
                all_positions_data.append(position_data)

            page_no += 1
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            logging.error(f"페이지 {page_no} 요청 실패: {e}")
            break
        except KeyError as e:
            logging.error(f"페이지 {page_no} JSON 파싱 에러 (키 에러): {e}")
            break
        except Exception as e:
            logging.error(f"예상치 못한 에러 발생: {e}")
            break

    return all_positions_data

In [ ]:
# 두 번째 셀을 실행하여 스크래핑된 데이터를 가져옵니다.
scraped_data = scrape_jumpit_positions()

In [ ]:
# 04. CSV 파일 생성
if not scraped_data:
    logging.warning("수집된 데이터가 없습니다. CSV 파일을 생성하지 않습니다.")
else:
    logging.info("수집된 데이터를 DataFrame으로 변환 중...")
    df = pd.DataFrame(scraped_data)

    # CSV 파일을 저장할 폴더 이름
    data_folder = 'data'

    # CSV 파일 이름 설정
    filename = f'data_jumpit_{job_category}.csv'

    # 저장할 전체 경로 생성
    filepath = os.path.join(data_folder, filename)

    # 해당 폴더가 없으면 생성
    if not os.path.exists(data_folder):
        try:
            os.makedirs(data_folder)
            logging.info(f"'{data_folder}' 폴더를 생성했습니다.")
        except OSError as e:
            logging.error(f"'{data_folder}' 폴더 생성 중 오류 발생: {e}", exc_info=True)
            print(f"\n폴더 생성 실패: {e}")
    else:
        logging.info(f"'{data_folder}' 폴더가 이미 존재합니다.")

    try:
        # encoding='utf-8-sig' : Excel에서 한글 깨짐 방지 (BOM 포함 UTF-8)
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        logging.info(f"DataFrame이 '{filepath}'으로 성공적으로 저장되었습니다.")
        print(f"\n파일 저장 완료: {filepath}")
    except Exception as e:
        logging.error(f"DataFrame을 CSV로 저장하는 중 오류 발생: {e}", exc_info=True)
        print(f"\n파일 저장 실패: {e}")

In [ ]:
df_read = pd.read_csv(f"data/{filename}", encoding='utf-8-sig')
df_read